# 04: Point Localization

*Authors: Lars Nitzschke, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Data Structures    
    3.1. Slab Decomposition  
    3.2. Vertical Decomposition
4. References  

## 1. Introduction

We start with the definition of the **point localization problem**. In the lecture it was stated as follows: Preprocess a planar subdivision such that for any query point _q_, the face of the subdivision containing _q_ can be given efficiently.
A **planar subdivision** is a partition of the plane by a a sert of non-crossing line segments into vertices edges and faces. The question at hand is a data structuring question, so we are interested in the **query time**, the **storage requirements** and the **preprocessing time**.

See the following two images for examples. They show the slab decomposition and the vertical decomposition respectively (on slightly different instances). The input **doubly-connected edge list** is coloured <font color='orange'>orange</font>, while the slab / vertical decomposition is marked in <font color='blue'>blue</font>.

<img style='float: left;' src='./images/04-image00.png' width='440px' height='412'>
<img style='float: left;' src='./images/04-image01.png' width='440px' height='412'>

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [ ]:
# Python standard library imports
import math
from abc import ABC, abstractmethod
from typing import Any, Optional, Iterable
from itertools import combinations, chain
import random

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTree, BinaryTreeDict, Comparator, ComparisonResult as CR, DoublyConnectedSimplePolygon, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict, PLSearchStructure
from modules.geometry import Point, PointReference, PointSequence, LineSegment, Orientation as ORT, EPSILON, Rectangle, VerticalOrientation as VORT, HorizontalOrientation as HORT
from modules.visualisation import VisualisationTool, PointLocationInstance, SlabDecompositionMode, VerticalExtensionMode, PointLocationMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, PointLocationInstance(drawing_epsilon=10, random_seed=42))
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.9 * c
s = c - r
t = c + r
u = (t - s) / 36

points = [Point(s + 20*u, t -  1*u), Point(s + 25*u, t -  6*u), Point(s + 30*u, t -  9*u), Point(s + 28*u, t - 11*u),
          Point(s + 19*u, t - 14*u), Point(s + 30*u, t - 21*u), Point(s + 28*u, t - 29*u), Point(s + 20*u, t - 33*u),
          Point(s + 15*u, t - 31*u), Point(s + 11*u, t - 34*u), Point(s +  2*u, t - 30*u), Point(s + 11*u, t - 24*u),
          Point(s +  5*u, t - 22*u), Point(s +  2*u, t - 17*u), Point(s +  5*u, t - 14*u), Point(s +  5*u, t -  6*u),
          Point(s + 11*u, t -  3*u), Point(s + 15*u, t - 17*u), Point(s + 19*u, t - 27*u)]
          
edges = [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8), (8,9), (9,10), (10,11), (11,12), (12,13), (13,14),
         (14,15), (15,16), (16,0), (1,3), (4,16), (4,17), (12,17), (14,17), (5,11), (6,18), (10,18)]

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Course Example 1", dcel)

points = [Point(s + 20*u, t -  1*u), Point(s + 25*u, t -  6*u), Point(s + 31*u, t -  9*u), Point(s + 28*u, t - 11*u),
          Point(s + 19*u, t - 14*u), Point(s + 30*u, t - 21*u), Point(s + 27*u, t - 29*u), Point(s + 22*u, t - 33*u),
          Point(s + 13*u, t - 31*u), Point(s +  8*u, t - 34*u), Point(s +  2*u, t - 30*u), Point(s + 11*u, t - 24*u),
          Point(s +  5*u, t - 22*u), Point(s +  1*u, t - 17*u), Point(s +  4*u, t - 14*u), Point(s +  6*u, t -  6*u),
          Point(s + 10*u, t -  3*u), Point(s + 15*u, t - 17*u), Point(s + 18*u, t - 27*u)]

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Course Example 2", dcel)

points = [Point(40,260), Point(180,210), Point(250,150), Point(340,160), Point(330, 70), Point(230,40), Point(180,50), Point(140,30), Point(100,50), Point(110,170)]

edges = [(0,9), (1,2), (3,4), (2,9), (8,9), (0,1), (6,7), (2,3), (2,6), (5,6), (4,5), (7,8)]

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Testing", dcel)

visualisation_tool.display()

## 3. Algorithms

In this notebook the first take a look at the simple **slab decomposition** and afterwards we approach the more sophisticated **vertical decomposition**.

### 3.1 Slab Decomposition

The first step in the lecture towards the **vertical decompostion** was the **slab decomposition**. Essentially the problem is reduced to a one-dimensional problem, which can efficiently be solved using a balanced binary search tree. To reduce the problem we draw vertical lines through all vertices, which divides the instance into a set of vertical strips, also called slabs. Inside each strip there is a well-defined _bottom-to-top order_ on the line segments. Each strip is therefore a one dimensional problem in _y_-direction. If we know between which line segments of the strip our point is, we found the face of the subdivision containing the point. Finding the correct strip is also a one dimensional problem, this time in _x_-direction.

Both one-dimensional problems are solved using a binary tree. (More information on the used binary trees can be found in [notebook no. 00](./00-Basics.ipynb).)
We start with the comparators for our trees. Same as in the previous notebooks we use a small epsilon in the comparison to somewhat compensate for the inaccuracy of floating-point arithmetic. (See [notebook no. 00](./00-Basics.ipynb) for further information on this as well.)

In [ ]:
class PointXComparator(Comparator[Point]):
    def compare(self, item: Any, key: Point) -> CR:
        if not isinstance(item, Point):
            raise TypeError("This comparator can only compared points to points")
        elif abs(item.x - key.x) < EPSILON:
            return CR.MATCH
        elif item.x - key.x > EPSILON:
            return CR.AFTER
        else:
            return CR.BEFORE
        
class LineSegmentYComparator(Comparator[LineSegment]):
    def compare(self, item: Any, key: LineSegment) -> CR:
        if isinstance(item, Point):
            if abs(item.y - key.y_from_x(item.x)) < EPSILON:
                return CR.MATCH
            elif item.y - key.y_from_x(item.x) > EPSILON:
                return CR.AFTER
            else:
                return CR.BEFORE
        elif isinstance(item, LineSegment):
            if abs(item.y_from_x(key.left.x) - key.left.y) < EPSILON and abs(item.y_from_x(key.right.x) - key.right.y) < EPSILON:
                return CR.MATCH
            elif item.left.y - key.left.y > EPSILON \
                or abs(item.y_from_x(key.left.x) - key.left.y) < EPSILON and item.y_from_x(key.right.x) - key.right.y > EPSILON:
                return CR.AFTER
            else:
                return CR.BEFORE        
        else:
            raise TypeError("This comparator can only compare either points or line segments to line segments")

Using these comparators our data structure is constructed as follows. We build a <tt>AnimationBinaryTreeDict</tt> $\mathcal{T}$ on all points in _x_-order. With each point we store a subtree on the line segments crossing the slab left of the point in _y_-order. This subtree is a <tt>AnimationBinaryTreeDict</tt> as well and with each line segement we store the face lying directly above. Note that in [1, section 6.1] the points are stored in an array instead. This is equivalent when using binary search on the array.

The line segments crossing each slab are computed using the sweep line technique. At each point we regard the linesegments which start and end here (left to right) and keep track of all currently crossing line segments. In the degenerate case where two or more points have the same _x_-coordinate we do compute our currently crossing line segments at each point, but only add the last point of that _x_-coordinate to $\mathcal{T}$.

The bounding box and the point sequence are only used for the algorithm drawings and animations and can be ignored.

In [ ]:
class SlabDecomposition(PLSearchStructure):
    def __init__(self, dcel: DoublyConnectedEdgeList, bounding_box: Rectangle, epsilon = EPSILON):
        # Stuff for drawing the vertical extensions at each point
        self._bounding_box = bounding_box
        self._point_sequence = PointSequence()
        # The outer face of the dcel is necessary when a search point does not lie between two line segments.
        self._ubounded_face = dcel.outer_face
        # Our actual tree structure
        self._x_comparator = PointXComparator()
        self._y_comparator = LineSegmentYComparator()
        self.search_tree: AnimationBinaryTreeDict[Point, AnimationBinaryTreeDict[LineSegment, Face]] = AnimationBinaryTreeDict(self._x_comparator, lambda p: p)

        currently_crossing_edges: list[HalfEdge] = []  # TODO: Maybe make this a tree.

        sorted_points = sorted(dcel.vertices, key = lambda vertex: vertex.point.x)
        iterator = iter(sorted_points)
        next_vertex = next(iterator, None)

        while next_vertex is not None:
            vertex = next_vertex
            next_vertex = next(iterator, None)

            if vertex.outgoing_edges() == []:  # Single vertices (i.e. no outgoing edges) are ignored as they lie completely inside one face.
                continue
            
            # Keep track of currently crossing line segments
            currently_crossing_edges = list(filter(lambda edge: edge.right.x - vertex.x > epsilon, currently_crossing_edges))
            currently_crossing_edges.extend(filter(lambda edge: edge.right.x - vertex.x > epsilon, vertex.outgoing_edges()))

            if next_vertex is not None and abs(vertex.x - next_vertex.x) < epsilon:  # Degenerate case: same x-coordinate
                self._point_sequence.append(vertex.point)
                continue

            # Build tree on the currently crossing line segments in y-direction
            subtree = AnimationBinaryTreeDict(self._y_comparator, lambda ls: PointReference([ls.left, ls.right], 0))
            for edge in currently_crossing_edges:
                subtree.insert(LineSegment(edge.left, edge.right), edge.incident_face)

            self.search_tree.insert(vertex.point, subtree)
            self._point_sequence.append(PointReference([vertex.point, Point(vertex.point.x, self._bounding_box.lower), Point(vertex.point.x, self._bounding_box.upper)], 0))

    def query(self, point: Point) -> tuple[Face, PointSequence]:
        ps = PointSequence()
        ps.append(point)
        # Search in x direction for the correct slab
        x_pred, x_ps = self.search_tree.search_predecessor(point)
        ps = ps + x_ps
        if x_pred is None:  # The search point is in the unbounded slab on the left side
            ps.clear()
            ps.append(point)
            return self._ubounded_face, ps
        else:
            y_tree = x_pred[1]
            # Search inside the slab for the correct face
            y_pred, y_ps = y_tree.search_predecessor(point)
            ps = ps + y_ps
            if y_pred is None:  # The search point is in the unbounded slab on the right side or below the first line segment in this slab.
                ps.clear()
                ps.append(point)
                return self._ubounded_face, ps
            dcel_face = y_pred[1]
            
            ps.clear()
            ps.append(point)
            for point in dcel_face.outer_points():
                ps.append(point)
            return dcel_face, ps

The storage requirements of the **slab decompostion** is $O(n^2)$ spacem where $n$ is the number of line segments. We have a tree of size $O(n)$, as we could store all points in the tree. With each point we store a subtree of size $O(n)$ because we store the line segments crossing the slab. See the lecture slides or [1, section 6.1] for an example using worst case storage.

Querying a point in our finished data structure then works as follows: Firstly we search in _x_-direction for the predecessor of our search point in our tree $\mathcal{T}$.
The subtree $\mathcal{T}_{y}$ stored with this predecessor represents the slab lying to its right. If we do not find a predecessor our search point lies in the far left unbounded face.
The outer dcel face is returned. Secondly we search for the predecessor of our search point in the subtree $\mathcal{T}_{y}$. The resulting line segment lies directly below the search point.
The face stored with the line segment is the face containing our search point. Thus we return the face.
If our search in the subtree $\mathcal{T}_{y}$ does not return a line segment our search point lies either in the unbounded face on the right or below the first line segment of that slab.
Again, we return the outer dcel face.

The query time for the data structure is good: We perform a search in the main tree $\mathcal{T}$ and a search in the subtree $\mathcal{T}_{y}$. Hence the query time is $O(\text{log}\; n)$.

Let's register both the construction of our slab decomposition and the point location query for visualisation.

In [ ]:
# Construction of the slab decomposition
def build_slab_decompostion(pl: PointLocation) -> PointSequence:
    pl._search_structure = SlabDecomposition(pl._dcel, bounding_box = Rectangle(Point(0, 0), Point(visualisation_tool._width, visualisation_tool._height)))
    return pl._search_structure._point_sequence

visualisation_tool.register_algorithm("Build Slab Decomp.", build_slab_decompostion, VerticalExtensionMode())

# Point location in the slab decompostion
def slab_point_location(pl: PointLocation) -> PointSequence:
    # Get search point from dcel
    point = pl._dcel._last_added_vertex.point
    point_edges = pl._dcel.find_edges_of_vertex(pl._dcel._last_added_vertex)
    if len(point_edges) > 1 or point_edges[0].origin != point_edges[0].destination:
        raise ValueError(f"Point Location needs to search for a single point not connected to any other point.")
    # Query the point
    face, point_sequence = pl._search_structure.query(point)
    return point_sequence

visualisation_tool.register_algorithm("Slab Point Location", slab_point_location, PointLocationMode(), preprocessing=build_slab_decompostion)

Now you can interactively test the algorithm and watch animations of it!
You need to run the notebook cells up until the following one for this purpose.
If you haven't used our interactive visualisation tool before, see [notebook no. 00](./00-Basics.ipynb) for an explanation.

For the the slab point location you need to add a search point to the instance by clicking the canvas. The query will always use the latest added point.
You can add edges to the instance by clicking an existing vertex as the first endpoint and for the second endpoint you can either click another existing vertex or create a new vertex by clicking clicking the canvas.

In [ ]:
visualisation_tool.display()

__*Takeaway:*__

* The slab decomposition is easy to implement and has a good query time, but its storage requirements make it a rather uninteresting data structure.

### 3.2 Vertical Decomposition

The **vertical decomposition** or **trapezoidal map** $\mathcal{T}$ is an improvement upon the slab decomposition. It uses less storage while maintaining the fast query time.
We call two line segments _non-crossing_ if their intersection is either empty or a common endpoint. Vertical decompositions are defined on a set $S$ of $n$ non-corssing segments in the plane.
Same as in the lecture we make a simplification. We get rid of unbounded faces at the boundary of the scene. We do this by introducing an axis-aligned bounding box $R$ containing all segments of $S$.

The vertical decompostion is then similar to the slab decomposition. We draw a vertical extension through every vertex. The difference is that we stop at the next line segment and do not draw up to the bounding box.
Therefore we cannot simply store the data structure in trees, but need a more direct approach. Our implementation uses the structure described in the lecture and in [1, section 6.1].
We have the class <tt>VDLineSegment</tt> which is the same as the normal line segment but additionally has a reference for the above lying dcel face.
Additionally we have the class <tt>VDFace</tt> which represents the trapezoids forming the trapezoidal map. It has references to its top and bottom line segments, its left and right bounding points and its up to four neighboring faces.
<tt>VDLeaf</tt> is not defined yet, but will be later, so do not worry about this for now.

In [ ]:
class VDLineSegment(LineSegment):
    def __init__(self, p: Point, q: Point):
        super().__init__(p, q)
        self.above_face: Optional[Face] = None  # The original face of the planar subdivision (DCEL)

    @classmethod
    def from_line_segment(cls, line_segment: LineSegment) -> VDLineSegment:
        return VDLineSegment(line_segment.left, line_segment.right)
    

class VDFace:
    def __init__(self, top_ls: VDLineSegment, bottom_ls: VDLineSegment, left_point: Point, right_point: Point) -> None:
        self.top_line_segment: VDLineSegment = top_ls
        self.bottom_line_segment: VDLineSegment = bottom_ls
        self.left_point: Point = left_point
        self.right_point: Point = right_point
        self.search_leaf: 'VDLeaf' = None
        self._neighbors: list['VDFace'] = [None, None, None, None]  # Up to four neighbors for each face

    # region neighbor properties

    @property
    def neighbors(self) -> list['VDFace']:  # Forward Referencing
        return self._neighbors
    
    @neighbors.setter
    def neighbors(self, new_neighbors: list[Optional['VDFace']]):
        if len(new_neighbors) != 4:
            raise Exception(f"Need exactly four optional faces for setting neighbors.")
        self.upper_right_neighbor = new_neighbors[0]
        self.upper_left_neighbor = new_neighbors[1]
        self.lower_left_neighbor = new_neighbors[2]
        self.lower_right_neighbor = new_neighbors[3]
    
    @property
    def upper_right_neighbor(self):
        return self._neighbors[0]
    
    @upper_right_neighbor.setter
    def upper_right_neighbor(self, new_neighbor: 'VDFace'):
        self._neighbors[0] = new_neighbor
        if new_neighbor is not None:
            new_neighbor._neighbors[1] = self
            
    @property
    def upper_left_neighbor(self):
        return self._neighbors[1]
    
    @upper_left_neighbor.setter
    def upper_left_neighbor(self, new_neighbor: 'VDFace'):
        self._neighbors[1] = new_neighbor
        if new_neighbor is not None:
            new_neighbor._neighbors[0] = self
    
    @property
    def lower_left_neighbor(self):
        return self._neighbors[2]
    
    @lower_left_neighbor.setter
    def lower_left_neighbor(self, new_neighbor: 'VDFace'):
        self._neighbors[2] = new_neighbor
        if new_neighbor is not None:
            new_neighbor._neighbors[3] = self
    
    @property
    def lower_right_neighbor(self):
        return self._neighbors[3]
    
    @lower_right_neighbor.setter
    def lower_right_neighbor(self, new_neighbor: 'VDFace'):
        self._neighbors[3] = new_neighbor
        if new_neighbor is not None:
            new_neighbor._neighbors[2] = self
    
    # endregion

The construction algorithm for the vertical decomposition $\mathcal{T}$ also builds a search structure $\mathcal{D}$ for point location queries. The search structure is a DAG ("directed acyclic graph") which consists of three types of nodes.
<tt>VDNode</tt> is the abstract base class defining the general structure of a node.

The first type of node is the leaf <tt>VDLeaf</tt>. It corresponds to one trapezoid in the vertical decomposition $\mathcal{T}$. The two other types of nodes are inner nodes. At each node we make a decision regarding the position of a search point $q$, that brings us closer to the trapezoid that contains $q$. When we reach a leaf our point $q$ can only lie in the corresponding trapezoid.

For decisions in $x$-direction we have <tt>VDXNode</tt>, which holds an endpoint $p$ of a line segment.
We ask the following question: "Does the given point $q$ lie to the left or to the right of the vertical line through $p$?" [1, section 6.2]
Here it is important to make sure the algorithm works as intended on degenerate cases, which are points with the same $x$-coordinate. As described in [1, section 6.3] this can be done by using a symbolic shear transform. For the horizontal orientation test this is equivalent to using the lexicographical order on the point, i.e. if the $x$-coordinate of the points is the same, their horizontal order is defined by their $y$-coordinate.

For decisions in y-direction we have <tt>VDYNode</tt>, which holds a line segment $s$.
At such a node we ask the following question: "Does the given point $q$ lie above or below the line segment $s$?" [1, section 6.2] It will be ensured that whenever we reach such a $y$-node that $q$ lies between the two endpoint of the line segment in horizontal order.
Again, we need to look at the degenerate cases. The line segment stored at the node could be a vertical segment. As $q$ lies between the line segment's enpoints in horizontal order, we know that $q$ then lies on the line segment.

When inserting a new line segment into our vertical decomposition it can happen that the new line segment shares it's left endpoint with another line segment stored at a $y$-node. For this case, during insertion, we not only search for the (left) endpoint but also pass the line segment along with it. If the case occures we then compare the slopes of the two line segments to determine wether to continue in the "upper"- or "lower"-child.

Finally, the point sequence which we also pass along is only for animations and can be ignored.

In [ ]:
class VDNode(ABC):
    def __init__(self) -> None:
        self._left: Optional['VDNode'] = None  # left/lower
        self._right: Optional['VDNode'] = None  # right/upper
        self._parents: list['VDNode'] = []  # The search structure is a DAG, hence a node can have multiple parents
    
    # region parent properties

    @property
    def parents(self) -> list['VDNode']:
        return self._parents
    
    @property
    def parent(self) -> Optional['VDNode']:
        if self._parents == []:
            return None
        return self._parents[0]

    @parent.setter
    def parent(self, parent: Optional['VDNode']):
        if parent is None or parent in self._parents:
            return
        self._parents.append(parent)        

    # endregion

    @abstractmethod
    def search(self, point: Point, line_segment: Optional[VDLineSegment] = None, point_sequence = PointSequence()) -> VDFace:
        pass

    def replace_with(self, new_node: 'VDNode') -> bool:
        if self.parents == []:  # node is the root
            return False
        for parent in self._parents:
            if parent._left is self:
                parent._left = new_node
            elif parent._right is self:
                parent._right = new_node
            else:
                raise Exception(f"{self} needs to be a child of its parent {parent}")
        
        new_node._parents = self.parents
        self._parents = []  
        return True

class VDLeaf(VDNode):
    def __init__(self, face: VDFace) -> None:
        super().__init__()
        self.face: VDFace = face
        self.face.search_leaf = self

    def search(self, point: Point, line_segment: Optional[VDLineSegment] = None, point_sequence = PointSequence()) -> VDFace:
        return self.face


class VDXNode(VDNode):
    def __init__(self, point: Point) -> None:
        super().__init__()
        self._point: Point = point

    # region properties

    @property
    def left(self) -> Optional[VDNode]:
        return self._left
    
    @left.setter
    def left(self, left: Optional[VDNode]):
        self._left = left
        if left != None:
            left.parent = self

    @property
    def right(self) -> Optional[VDNode]:
        return self._right
    
    @right.setter
    def right(self, right: Optional[VDNode]):
        self._right = right
        if right != None:
            right.parent = self

    # endregion

    def search(self, point: Point, line_segment: Optional[VDLineSegment] = None, point_sequence = PointSequence()) -> VDFace:
        point_sequence.append(self._point)
        if point.horizontal_orientation(self._point) == HORT.LEFT:  # Using symbolic shear transform
            return self._left.search(point, line_segment, point_sequence)
        else:  # The point lies to the right or coincides with the point of the node. Then we decide to continue to the right ([1], page 130 last paragraph). Durint insertion this results in a trapezoid of 0 width.
            return self._right.search(point, line_segment, point_sequence)


class VDYNode(VDNode):
    def __init__(self, line_segment: VDLineSegment) -> None:
        super().__init__()
        self._line_segment: VDLineSegment = line_segment

    # region properties

    @property
    def lower(self) -> Optional[VDNode]:
        return self._left
    
    @lower.setter
    def lower(self, lower: Optional[VDNode]):
        self._left = lower
        if lower != None:
            lower.parent = self

    @property
    def upper(self) -> Optional[VDNode]:
        return self._right
    
    @upper.setter
    def upper(self, upper: Optional[VDNode]):
        self._right = upper
        if upper != None:
            upper.parent = self

    # endregion

    def search(self, point: Point, line_segment: Optional[VDLineSegment] = None, point_sequence = PointSequence()) -> VDFace:
        point_sequence.append(PointReference([self._line_segment.left, self._line_segment.right], 0))
        
        cr = point.vertical_orientation(self._line_segment)
        # During insertion ('line_segment != None'): cr == VORT.ON can only happen if the new line segment shares its left endpoint with the line segment stored at this node
        if cr == VORT.ABOVE or (cr == VORT.ON and (line_segment is None or line_segment.slope() > self._line_segment.slope())):
            return self.upper.search(point, line_segment, point_sequence)
        elif cr == VORT.BELOW or (cr == VORT.ON and line_segment.slope() < self._line_segment.slope()):
            return self.lower.search(point, line_segment, point_sequence)
        else:
            raise AttributeError(f"The line segment {line_segment} cannot be inserted because it is already in the vertical decomposition")

We now start with the construction algorithm of the vertical decomposition. The algorithm in incremental, so we insert all line segments iteratively.
For each line segment we proceed as follows: In a first step we update the actual vertical decomposition $\mathcal{T}$ and using that we then update the search structure $\mathcal{D}$.
The vertical decomposition $\mathcal{T}$ is initialised with the initial trapezoid which corresponds to the bounding box.

Updating the vertical decomposition $\mathcal{T}$ is quite intricate. Here will only give a general overview. Some details are explained in the comments in the code and more information can be found in [1, chapter 6].
The updating starts with the line segment and the face in which its left endpoint lies. It returns all trapezoids that are now unvalid and all new or changed trapezoids. A trapezoid may occure in both lists.

First we find for all trapezoids that are intersected by the line segement by traversing through the neighbors. If the line segment lies completely in one trapezoid we can simply partition that trapezoid into up to four new trapezoids, making sure all neighbors are set correctly. It is important to make sure that the new line segment's left and right endpoint could be the same as the left and right bounding points of the containing trapezoid (See method <tt>_partition_trapezoid(...)</tt>).
In the more complicated case, both trapezoids containing the left and right endpoints of the line segment are partitioned. All other trapezoids intersected by the line segment need to be regarded as well. They are shrinked in y-direction up to the line segment and the area on the other side of the line segment is merged with the trapezoid to the left.

In [ ]:
class VerticalDecomposition:
    def __init__(self, bounding_box: Rectangle, dcel: DoublyConnectedEdgeList) -> None:
        self._point_sequence: PointSequence = PointSequence()  # For drawings

        # Build the initial trapezoid
        top = VDLineSegment(Point(bounding_box.left, bounding_box.upper), Point(bounding_box.right, bounding_box.upper))
        bottom = VDLineSegment(Point(bounding_box.left, bounding_box.lower), Point(bounding_box.right, bounding_box.lower))
        bottom._above_dcel_face = dcel.outer_face
        
        self.initial_trapezoid = VDFace(top, bottom, top.left, top.right)

    def update(self, line_segment: VDLineSegment, left_point_face: VDFace) -> tuple[list[VDFace], list[Optional[VDFace]]]:
        self._point_sequence.animate(PointReference([line_segment.left, line_segment.right], 0))

        # Find all other k intersected trapezoids (via neighbors) in O(k) time (see [1], page 130)
        intersected_trapezoids: list[VDFace] = []  # Does not include the left_point_face
        last_intersected = left_point_face
        while line_segment.right.horizontal_orientation(last_intersected.right_point) == HORT.RIGHT:  # ls extends to the right of the last added trapezoid
            if last_intersected.right_point.vertical_orientation(line_segment) == VORT.BELOW:
                last_intersected = last_intersected.upper_right_neighbor
            else:  # vertical_orientation == VORT.ABOVE, VORT.ON is impossible
                last_intersected = last_intersected.lower_right_neighbor
            intersected_trapezoids.append(last_intersected)
            
        if intersected_trapezoids == []:  # Simple case: the "line_segment" is completely contained in the trapezoid "left_point_face"
            # The trapezoid is replaced by up to four new trapezoids (see [1], section 6.2)
            return [left_point_face], list(self._partition_trapezoid(left_point_face, line_segment))

        # Other (more complicated) case: the "line_segment" intersects two or more trapezoids (Update in O(k) time)
        right_point_face = intersected_trapezoids.pop()
        
        # Partition the first and last trapezoid into three new trapezoids each
        lpf_upper_right_neighbor, lpf_lower_right_neighbor = left_point_face.upper_right_neighbor, left_point_face.lower_right_neighbor  # Retain pointers to the neighbors before any changes
        rpf_upper_left_neighbor, rpf_lower_left_neighbor = right_point_face.upper_left_neighbor, right_point_face.lower_left_neighbor

        left_face, left_face_above, left_face_below, _ = self._partition_trapezoid(left_point_face, line_segment)

        if left_point_face.right_point.vertical_orientation(line_segment) == VORT.ABOVE:  # Setting the top/bottom most neighbor
            left_face_above.upper_right_neighbor = lpf_upper_right_neighbor
        elif left_point_face.right_point.vertical_orientation(line_segment) ==  VORT.BELOW:
            left_face_below.lower_right_neighbor = lpf_lower_right_neighbor
        else:
            raise RuntimeError(f"Point {left_point_face.right_point} must not lie on line induced by the line segment {line_segment}")

        _, right_face_above, right_face_below, right_face = self._partition_trapezoid(right_point_face, line_segment)

        # Shorten vertical extensions that abut on the LS. => Merge trapezoids along the line-segment. (see [1], page 132)
        last_face_above, last_face_below = left_face_above, left_face_below
        for trapezoid in intersected_trapezoids:
            last_face_above, last_face_below = self._merge_trapezoids(trapezoid, line_segment, last_face_above, last_face_below)

        # Merge with (already split) last trapezoid
        self._point_sequence.animate(right_face_above.left_point)
        seq_point = self._point_sequence[right_face_above.left_point].copy()
        if right_face_above.left_point.vertical_orientation(line_segment) == VORT.ABOVE:  # point is the original leftp of right_point_face (equal in right_face_below)
            # Merge trapezoids below the LS and discard right_face_below
            last_face_below.lower_right_neighbor = right_face_below.lower_right_neighbor  # the right_point_face if the right endpoint of the linesegment is new and its old lower_right_neighbor otherwise
            last_face_below.upper_right_neighbor = None
            last_face_below.right_point = line_segment.right
            
            last_face_above.lower_right_neighbor = right_face_above  # Connect neighboring trapezoids above the LS
            right_face_above.upper_left_neighbor = rpf_upper_left_neighbor  # Connect even higher neighbor
            kept_face = right_face_above
            if isinstance(seq_point, PointReference):
                seq_point.container[2] = Point(right_face_above.left_point.x, line_segment.y_from_x(right_face_above.left_point.x))
        elif right_face_above.left_point.vertical_orientation(line_segment) == VORT.BELOW:
            # Merge trapezoids above the LS and discard right_face_above
            last_face_above.lower_right_neighbor = None
            last_face_above.upper_right_neighbor = right_face_above.upper_right_neighbor
            last_face_above.right_point = line_segment.right
            
            last_face_below.upper_right_neighbor = right_face_below  # Connect neighboring trapezoids below the LS
            right_face_below.lower_left_neighbor = rpf_lower_left_neighbor
            kept_face = right_face_below
            if isinstance(seq_point, PointReference):
                seq_point.container[1] = Point(right_face_above.left_point.x, line_segment.y_from_x(right_face_above.left_point.x))
        else:
            raise RuntimeError(f"Point {right_face_above.left_point} must not lie on line induced by the line segment {line_segment}")
        
        self._point_sequence[right_face_above.left_point] = seq_point        
        return [left_point_face] + intersected_trapezoids + [right_point_face], [left_face, left_face_above, left_face_below, kept_face, right_face]
    
    def _partition_trapezoid(self, face: VDFace, line_segment: VDLineSegment) -> tuple[Optional[VDFace], VDFace, VDFace, Optional[VDFace]]:
        trapezoid_top = VDFace(face.top_line_segment, line_segment, line_segment.left, line_segment.right)
        trapezoid_bottom = VDFace(line_segment, face.bottom_line_segment, line_segment.left, line_segment.right)
        if (hort := line_segment.left.horizontal_orientation(face.left_point)) == HORT.LEFT:  # Case ls extends further to the left
            trapezoid_left = None
            trapezoid_top.left_point = face.left_point
            trapezoid_bottom.left_point = face.left_point
        elif hort == HORT.EQUAL:   # Case where the new linesegment shares and enpoint with an already existing endpoint
            trapezoid_left = None
            trapezoid_top.upper_left_neighbor = face.upper_left_neighbor
            trapezoid_bottom.lower_left_neighbor = face.lower_left_neighbor
            self._point_sequence.animate(line_segment.left)
        else:
            trapezoid_left = VDFace(face.top_line_segment, face.bottom_line_segment, face.left_point, line_segment.left)
            trapezoid_left.neighbors = [trapezoid_top, face.upper_left_neighbor, face.lower_left_neighbor, trapezoid_bottom]
            point_above = Point(line_segment.left.x, face.top_line_segment.y_from_x(line_segment.left.x))
            point_below = Point(line_segment.left.x, face.bottom_line_segment.y_from_x(line_segment.left.x))
            self._point_sequence.append(PointReference([line_segment.left, point_above.copy(), point_below.copy()], 0))  # For animations: copy(), because the values might change later.
        
        if (hort := line_segment.right.horizontal_orientation(face.right_point)) == HORT.RIGHT:
            trapezoid_right = None
            trapezoid_top.right_point = face.right_point
            trapezoid_bottom.right_point = face.right_point
        elif hort == HORT.EQUAL:
            trapezoid_right = None
            trapezoid_top.upper_right_neighbor = face.upper_right_neighbor
            trapezoid_bottom.lower_right_neighbor = face.lower_right_neighbor
            self._point_sequence.animate(line_segment.right)
        else:
            trapezoid_right = VDFace(face.top_line_segment, face.bottom_line_segment, line_segment.right, face.right_point)
            trapezoid_right.neighbors = [face.upper_right_neighbor, trapezoid_top, trapezoid_bottom, face.lower_right_neighbor]
            point_above = Point(line_segment.right.x, face.top_line_segment.y_from_x(line_segment.right.x))
            point_below = Point(line_segment.right.x, face.bottom_line_segment.y_from_x(line_segment.right.x))
            self._point_sequence.append(PointReference([line_segment.right, point_above.copy(), point_below.copy()], 0))

        return trapezoid_left, trapezoid_top, trapezoid_bottom, trapezoid_right
    
    def _merge_trapezoids(self, trapezoid: VDFace, line_segment: VDLineSegment, last_above: VDFace, last_below: VDFace):
        self._point_sequence.animate(trapezoid.left_point)
        seq_point = self._point_sequence[trapezoid.left_point].copy()
        if trapezoid.left_point.vertical_orientation(line_segment) == VORT.ABOVE:
            # Merge trapezoids below the LS
            if trapezoid.right_point.vertical_orientation(line_segment) == VORT.BELOW:  # Next one is on the other side, new neighbors are correct and final
                last_below.lower_right_neighbor = trapezoid.lower_right_neighbor
                last_below.upper_right_neighbor = trapezoid.upper_right_neighbor
            last_below.right_point = trapezoid.right_point

            # Shrink the intersected trapezoid to be above the LS
            trapezoid.bottom_line_segment = line_segment
            trapezoid.lower_left_neighbor = last_above
            last_above = trapezoid
            if isinstance(seq_point, PointReference):
                seq_point.container[2] = Point(trapezoid.left_point.x, line_segment.y_from_x(trapezoid.left_point.x))
        elif trapezoid.left_point.vertical_orientation(line_segment) == VORT.BELOW:
            # Merge trapezoids above the LS
            if trapezoid.right_point.vertical_orientation(line_segment) == VORT.ABOVE:
                last_above.lower_right_neighbor = trapezoid.lower_right_neighbor
                last_above.upper_right_neighbor = trapezoid.upper_right_neighbor
            last_above.right_point = trapezoid.right_point

            # Shrink the intersected trapezoid to be below the LS
            trapezoid.top_line_segment = line_segment
            trapezoid.upper_left_neighbor = last_below
            last_below = trapezoid
            if isinstance(seq_point, PointReference):
                seq_point.container[1] = Point(trapezoid.left_point.x, line_segment.y_from_x(trapezoid.left_point.x))
        else:
            raise RuntimeError(f"Point {trapezoid.left_point} must not lie on line induced by the line segment {line_segment}")
        self._point_sequence[trapezoid.left_point] = seq_point
        return last_above, last_below

With the vertical decomposition updated we can shift our focus to the update of the search structure <tt>VDSearchStructure</tt>.
From updating the vertical decomposition we already have all intersected, no unvalid, trapezoids and therefore all unvalid leafs.
In the search structure we need to replace those leafes. Additionally we already have all new trapezoids which provide us with new leafs for the search structure.

Again, we have the simply case where the new line segment lies completely in an already existing trapezoid.
In this case we can replace the corresponding leaf by a small tree consisting of a $y$-node and up to two $x$-nodes.
In the more complex case both leafs corresponding to the left and right subtree are replaced by a small tree containing
a $y$-node and possibly a $x$-node. All other unvalid leafs are replaced by a single $y$-node.
The leafs of all these small trees are the ones we got from the new trapezoids in the vertical decomposition.

In the <tt>VDSearchStructure</tt> we can also find a method for querying the search structure. It simply starts a search at the root of the structure.
The search than proceeds as described above in the introduction of the nodes the structure consists of.
Finally we look at the bottom line segment of our returned vertical decompostion face which has a reference to the face of the original DCEL.

In [ ]:
class VDSearchStructure(PLSearchStructure):
    def __init__(self, initial_face: VDFace) -> None:
        self.root: VDNode = VDLeaf(initial_face)

    def query(self, point: Point) -> tuple[Face, PointSequence]:
        """Query the search structure and search for the given point"""
        point_sequence = PointSequence()  # For drawings
        point_sequence.append(point)

        vd_face = self.root.search(point, point_sequence=point_sequence)
        dcel_face = vd_face.bottom_line_segment.above_face
        
        point_sequence.clear()
        point_sequence.append(point)
        for vertex in dcel_face.outer_vertices():
            point_sequence.append(vertex.point)
        return dcel_face, point_sequence
    
    def update(self, line_segment: VDLineSegment, unvalid_leafs: list[Optional[VDLeaf]], new_leafs: list[Optional[VDLeaf]]):
        """Update the search structure using known leafs from updating the vertical decomposition"""
        if len(unvalid_leafs) == 1:  # Simple case: the "line_segment" is completely contained in the one trapezoid "left_point_face"
            tree = self._build_subtree(line_segment, new_leafs[0:4])

            success = unvalid_leafs[0].replace_with(tree)
            if not success:  # root element
                self.root = tree
            return
        
        # Other (more complicated) case: the line segment intersects two or more trapezoids.
        # Replace the leaf containing the left endpoint by a small tree composed of an x- and a y-node.
        left_endpoint_leaf = unvalid_leafs.pop(0)
        tree = self._build_subtree(line_segment, new_leafs[0:3])
        left_endpoint_leaf.replace_with(tree)  # checking for success is not necessary. The leaf node "left_endpoint_leaf" cannot be the root as there are >= 2 total unvalid leafs

        right_endpoint_leaf = unvalid_leafs.pop()  # leaf containing the right endpoint, filled after treatment of intersected trapezoids

        opposite_leaf = None  # leaf corresponding to the trapezoid on the other side of the LS
        if not unvalid_leafs == []:  # left and right point do not lie in directly neighboring faces
            # Init opposite_leaf: Either the leaf for left_face_above or left_face_below
            if unvalid_leafs[0].face.bottom_line_segment is line_segment:
                opposite_leaf = new_leafs[2]
            else:
                opposite_leaf = new_leafs[1]
            opposite_leaf_orientation = opposite_leaf.face.right_point.vertical_orientation(line_segment)

            for i in range(0, len(unvalid_leafs)):  # Replace all unvalid leafs (beside first and last) by a single y-node
                tree = VDYNode(line_segment)
                unvalid_leafs[i].replace_with(tree)
                
                unvalid_leaf_orientation = unvalid_leafs[i].face.left_point.vertical_orientation(line_segment)
                if unvalid_leaf_orientation == opposite_leaf_orientation:
                    opposite_leaf = unvalid_leafs[i-1]
                    opposite_leaf_orientation = opposite_leaf.face.left_point.vertical_orientation(line_segment)

                if unvalid_leaf_orientation == VORT.ABOVE:
                    tree.upper = unvalid_leafs[i]
                    tree.lower = opposite_leaf
                else:  # VORT.BELOW
                    tree.upper = opposite_leaf
                    tree.lower = unvalid_leafs[i]

            if new_leafs[-2].face.left_point.vertical_orientation(line_segment) == opposite_leaf_orientation:
                opposite_leaf = unvalid_leafs[-1]  # Maintain opposite leaf one more time
        else:
            if new_leafs[-2].face.bottom_line_segment is line_segment:
                opposite_leaf = new_leafs[2]
            else:
                opposite_leaf = new_leafs[1]

        # Replace the leaf containing the right endpoint        
        if new_leafs[-2].face.bottom_line_segment is line_segment:  # kept_face is either right_face_above or right_face_below
            tree = self._build_subtree(line_segment, [None, new_leafs[-2], opposite_leaf, new_leafs[-1]])
        else:  # if new_leafs[-2].face.top_line_segment is line_segment:
            tree = self._build_subtree(line_segment, [None, opposite_leaf, new_leafs[-2], new_leafs[-1]])

        right_endpoint_leaf.replace_with(tree)

    def _build_subtree(self, line_segment: VDLineSegment, leafs: list[Optional[VDLeaf]]) -> VDNode:
        if len(leafs) < 3 or len(leafs) > 4:
            raise ValueError(f"The list of leafs {leafs} needs to be of length 3 or 4 to build a subtree.")
        # The leaf is replaced by a small tree composed of two x-nodes and a y-node
        tree = VDYNode(line_segment)
        tree.upper = leafs[1]
        tree.lower = leafs[2]
        if len(leafs) > 3 and leafs[3] is not None:  # The new line segment does not share an enpoint with an already existing endpoint
            child = tree
            tree = VDXNode(line_segment.right)
            tree.left = child
            tree.right = leafs[3]
        if leafs[0] is not None:
            child = tree
            tree = VDXNode(line_segment.left)
            tree.left = leafs[0]
            tree.right = child
        return tree

Now we can put it all together. For the construction of the vertical decompostion we need to proprocess out DCEL as we do not
need all the information stored by it. We only keep the line segments and with every line segmnet we store the dcel face lying directly above,
using our <tt>VDLineSegment.</tt>

As seen in the lecture and described in [1, section 6.2], we use randomized incremental construction for building the vertical decomposition.
The reason for this is that the order in which the line segments are added changes the search structure, specifically it's depth.
Therefore, some ordering could lead to good query times, while other ones lead to bad query times.
Instead of finding an suitable ordering we use random ordering, which leads to acceptable running times.
Here we introduce the randomization by shuffling the list of line segments. The seed is fixed so the algorithm works deterministic,
but the seed can be changed or even set to <tt>None</tt> at the top where the <tt>VisualistionTool</tt> is created.
After the segments are shuffled, they are incrementally inserted into our data structure. The procedure was already described above.

The construction algorithm is then registered in our <tt>VisualistionTool</tt>.

In [ ]:
# DCEL preprocessing for building the vertical decomposition
def dcel_prepocessing(dcel: DoublyConnectedEdgeList) -> Iterable[VDLineSegment]:
    segments: list[VDLineSegment] = []
    for edge in dcel.edges:
        if edge.origin == edge.destination:
            continue
        if edge.origin is edge.left_and_right[0]:  # Make sure only one of each Halfedges is used
            ls = VDLineSegment(edge.origin.point, edge.destination.point)
            ls.above_face = edge.incident_face
            segments.append(ls)
    return segments

# Construction of the vertical decomposition
def build_vertical_decomposition(pl: PointLocation) -> PointSequence:
    pl._vertical_decomposition = VerticalDecomposition(pl._bounding_box, pl._dcel)
    pl._search_structure = VDSearchStructure(pl._vertical_decomposition.initial_trapezoid)
    segments = dcel_prepocessing(pl._dcel)
    if pl._random_seed is not None:
        random.seed(pl._random_seed)
    random.shuffle(segments)  # Randomized Incremental Construction
    
    for line_segment in segments:
        # Search the trapezoid containing the left point
        left_point_face = pl._search_structure.root.search(line_segment.left, line_segment)

        # 1. Update the vertical decomposition
        unvalid_trapezoids, new_trapezoids = pl._vertical_decomposition.update(line_segment, left_point_face)

        # 2. Update the search structure
        unvalid_leafs = [trapezoid.search_leaf for trapezoid in unvalid_trapezoids]
        new_leafs = [VDLeaf(trapezoid) if trapezoid is not None else None for trapezoid in new_trapezoids]
        pl._search_structure.update(line_segment, unvalid_leafs, new_leafs)

    return pl._vertical_decomposition._point_sequence

visualisation_tool.register_algorithm("Build Vert. Decomp.", build_vertical_decomposition, VerticalExtensionMode())

For querying we use the latest added point, same as with the slab decompostion. The algorithm simply calls the <tt>VDSearchStructure</tt> and the procedure from there was already described. Naturally, we also register the querying algorithm in our <tt>VisualistionTool</tt>. 

In [ ]:
# Point location in the vertical decomposition
def vd_point_location(pl: PointLocation) -> PointSequence:
    point = pl._dcel._last_added_vertex.point
    point_edges = pl._dcel.find_edges_of_vertex(pl._dcel._last_added_vertex)
    if len(point_edges) > 1 or point_edges[0].origin != point_edges[0].destination:
        raise ValueError(f"Point Location needs to search for a single point not connected to any other point.")
    if not isinstance(pl._search_structure, VDSearchStructure):
        raise RuntimeError(f"The search structure needs to be a VDSearchStructure to run VD point location")
    dcel_face, ps = pl._search_structure.query(point)
    return ps

visualisation_tool.register_algorithm("Point Location", vd_point_location, PointLocationMode(), preprocessing=build_vertical_decomposition)

Now you can watch the algorithms in action:

In [ ]:
visualisation_tool.display()

__*Takeaways:*__

* The vertical decomposition is much more difficult to implement than the slab decomposition but improve the space requirements.

* The vertical decomposition has good query and construction time.

* **Randomized Incremental Construction** is a powerful tool when the order in which a data structure is constructed matters.

In [ ]:
# TODO: Animation of DCEL drawings

# TODO: DCEL check for crossings and raise exception or add as segment not as an edge
# TODO: Clicking the canvas sometimes does not work

# TODO: Add AnimationTrees and DCEL to notebook 00.
# TODO: Look into ETE Toolkit for tree visualisation

# TODO: Reset execution counter of cells and add images as default cell outputs.
# TODO: Worst case example of the slab decomposition